In [ ]:
! pip install transformers datasets

In [2]:
from datasets import load_dataset

dataset = load_dataset("../cnn_dailymail_suenes/cnn_dailymail_suenes.py")
print(dataset)

/home/jobayer/wsl-workspace/579X/SueNes/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No config specified, defaulting to: cnn_dailymail_suenes/1.0.0
Found cached dataset cnn_dailymail_suenes (/home/jobayer/.cache/huggingface/datasets/cnn_dailymail_suenes/1.0.0/1.0.0/066f2a33d28679b436a34f9ac68a5860f9a0a9ebfe420002c59c849e6dde9337)
100%|██████████| 3/3 [00:00<00:00, 421.92it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'score'],
        num_rows: 138502
    })
    validation: Dataset({
        features: ['text', 'summary', 'score'],
        num_rows: 17917
    })
    test: Dataset({
        features: ['text', 'summary', 'score'],
        num_rows: 17722
    })
})


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny", model_max_length=512)

def tokenize_function(examples):
    return tokenizer(examples["text"], examples["summary"], padding=True, truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

2022-11-13 18:11:37.433008: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-13 18:11:37.631425: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-13 18:11:37.631460: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-13 18:11:37.664960: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-13 18:11:38.363482: W tensorflow/stream_executor/platform/de

In [4]:
from transformers import DefaultDataCollator, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [5]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["score"],
    shuffle=True,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["score"],
    shuffle=False,
    batch_size=8,
)

2022-11-13 18:17:07.281607: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-13 18:17:07.282660: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-13 18:17:07.282783: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (COMS-SCO-097): /proc/driver/nvidia/version does not exist
2022-11-13 18:17:07.284942: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=1, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), # use regeression probleam loss
    metrics=tf.metrics.BinaryAccuracy(),
)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

Epoch 1/3
   32/17313 [..............................] - ETA: 2:35:29 - loss: 0.6946 - binary_accuracy: 0.0000e+00

KeyboardInterrupt: 